# PEFT 进阶操作

## 1. 自定义模型适配

In [1]:
import torch
from torch import nn
from peft import LoraConfig, get_peft_model, PeftModel

In [3]:
net1 = nn.Sequential(
    nn.Linear(10, 10),
    nn.ReLU(),
    nn.Linear(10, 2)
)
net1

Sequential(
  (0): Linear(in_features=10, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=2, bias=True)
)

In [4]:
for name, param in net1.named_parameters():
    print(name)

0.weight
0.bias
2.weight
2.bias


In [5]:
config = LoraConfig(target_modules=["0"])

In [6]:
model1 = get_peft_model(net1, config)

In [7]:
model1

PeftModel(
  (base_model): LoraModel(
    (model): Sequential(
      (0): lora.Linear(
        (base_layer): Linear(in_features=10, out_features=10, bias=True)
        (lora_dropout): ModuleDict(
          (default): Identity()
        )
        (lora_A): ModuleDict(
          (default): Linear(in_features=10, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (default): Linear(in_features=8, out_features=10, bias=False)
        )
        (lora_embedding_A): ParameterDict()
        (lora_embedding_B): ParameterDict()
      )
      (1): ReLU()
      (2): Linear(in_features=10, out_features=2, bias=True)
    )
  )
)

## 2. 多适配器加载与切换

In [8]:
net2 = nn.Sequential(
    nn.Linear(10, 10),
    nn.ReLU(),
    nn.Linear(10, 2)
)
net2

Sequential(
  (0): Linear(in_features=10, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=2, bias=True)
)

In [9]:
config1 = LoraConfig(target_modules=["0"])
model2 = get_peft_model(net2, config1)
model2.save_pretrained("./loraA")

In [10]:
config2 = LoraConfig(target_modules=["2"])
model2 = get_peft_model(net2, config2)
model2.save_pretrained("./loraB")

In [11]:
net2 = nn.Sequential(
    nn.Linear(10, 10),
    nn.ReLU(),
    nn.Linear(10, 2)
)
net2

Sequential(
  (0): Linear(in_features=10, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=2, bias=True)
)

In [12]:
model2 = PeftModel.from_pretrained(net2, model_id="./loraA/", adapter_name="loraA")
model2

PeftModel(
  (base_model): LoraModel(
    (model): Sequential(
      (0): lora.Linear(
        (base_layer): Linear(in_features=10, out_features=10, bias=True)
        (lora_dropout): ModuleDict(
          (loraA): Identity()
        )
        (lora_A): ModuleDict(
          (loraA): Linear(in_features=10, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (loraA): Linear(in_features=8, out_features=10, bias=False)
        )
        (lora_embedding_A): ParameterDict()
        (lora_embedding_B): ParameterDict()
      )
      (1): ReLU()
      (2): Linear(in_features=10, out_features=2, bias=True)
    )
  )
)

In [13]:
model2.load_adapter("./loraB/", adapter_name="loraB")
model2

PeftModel(
  (base_model): LoraModel(
    (model): Sequential(
      (0): lora.Linear(
        (base_layer): Linear(in_features=10, out_features=10, bias=True)
        (lora_dropout): ModuleDict(
          (loraA): Identity()
        )
        (lora_A): ModuleDict(
          (loraA): Linear(in_features=10, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (loraA): Linear(in_features=8, out_features=10, bias=False)
        )
        (lora_embedding_A): ParameterDict()
        (lora_embedding_B): ParameterDict()
      )
      (1): ReLU()
      (2): lora.Linear(
        (base_layer): Linear(in_features=10, out_features=2, bias=True)
        (lora_dropout): ModuleDict(
          (loraB): Identity()
        )
        (lora_A): ModuleDict(
          (loraB): Linear(in_features=10, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (loraB): Linear(in_features=8, out_features=2, bias=False)
        )
        (lora_embedding_A): ParameterDi

In [14]:
model2.active_adapter

'loraA'

In [15]:
model2(torch.arange(0, 10).view(1, 10).float())

tensor([[-1.1325,  0.9759]], grad_fn=<AddmmBackward0>)

In [16]:
for name, param in model2.named_parameters():
    print(name, param)

base_model.model.0.base_layer.weight Parameter containing:
tensor([[ 1.4645e-01,  7.1984e-02, -2.0617e-01, -2.6386e-01, -9.8245e-02,
          2.5730e-01, -9.7435e-02, -1.7213e-01,  2.5057e-01, -1.3699e-01],
        [-6.1466e-02, -2.6721e-01, -1.3274e-01,  4.7159e-02,  1.4495e-02,
          2.6219e-01, -1.2239e-01, -2.4726e-01,  1.4580e-01,  1.7786e-01],
        [-2.2877e-01, -9.2916e-02, -8.5293e-03, -1.2721e-01,  1.4724e-01,
          1.8251e-01,  3.0822e-01,  1.3948e-01,  6.8516e-02,  2.5817e-01],
        [-5.6962e-03,  1.4657e-01, -2.9567e-01,  2.8081e-01, -1.3695e-01,
         -1.1256e-01,  1.5793e-01,  2.8332e-01,  1.5539e-01, -2.7911e-01],
        [-2.5973e-01, -1.8877e-01,  2.3679e-01,  3.6140e-02, -1.9100e-01,
          1.5634e-01, -4.2791e-02,  2.2269e-01,  2.6091e-01, -8.2455e-02],
        [-1.4657e-01,  2.7296e-01,  9.0944e-02, -1.5734e-01,  4.5964e-02,
          2.9930e-01, -2.1646e-01,  2.8357e-01,  2.9085e-01,  2.6478e-01],
        [ 2.0233e-01,  3.1054e-05,  2.9408e-01,

In [17]:
for name, param in model2.named_parameters():
    if name in ["base_model.model.0.lora_A.loraA.weight", "base_model.model.0.lora_B.loraA.weight"]:
        param.data = torch.ones_like(param)

In [18]:
model2(torch.arange(0, 10).view(1, 10).float())

tensor([[108.5005, 215.6204]], grad_fn=<AddmmBackward0>)

In [19]:
model2.set_adapter("loraB")

In [20]:
model2.active_adapter

'loraB'

In [21]:
model2(torch.arange(0, 10).view(1, 10).float())

tensor([[-1.1325,  0.9759]], grad_fn=<AddBackward0>)

## 3. 禁用适配器

In [22]:
model2.set_adapter("loraA")

In [28]:
model2(torch.arange(0, 10).view(1, 10).float())

tensor([[108.5005, 215.6204]], grad_fn=<AddmmBackward0>)

In [29]:
with model2.disable_adapter():
    print(model2(torch.arange(0, 10).view(1, 10).float()))

tensor([[-1.1325,  0.9759]])
